In [ ]:
from pyscenic.utils import load_motifs
import pickle
import numpy as np
from arboreto.utils import load_tf_names
import pandas as pd
import statistics
from collections import Counter

In [ ]:
data_dir = '../../../Processed_Data'
scenic_results_dir = '../../../Results/Results_no_ptprc_adgre1/SCENIC_results'

# Format the regulons and save them as a txt file

In [ ]:
#Convert the regulons generated by pySCENIC to a dictionary containing TF: {(target:importance....)} 
def format_regulons(regulon_path,regulon_save_path):
    regulons =pickle.load(regulon_path)
    reg_dict = {}
    for reg in regulons:
        reg_dict[reg.name] = {key:np.round(reg.gene2weight[key],decimals=3) for key in reg.gene2weight}
    with open(regulon_save_path, "wb") as f:
        pickle.dump(reg_dict, f)
    return

#Save the formatted regulons as a txt file
def save_as_txt(regulon_path,output_file):
    # Open the file for writing
    regulon_dict = {}
    print(regulon_path)
    with open(regulon_path, "rb") as f:
        regulons =pickle.load(f)
    for reg in regulons:
        regulon_dict[reg.name] = {key:np.round(reg.gene2weight[key],decimals=3) for key in reg.gene2weight}
    with open(output_file, "w") as f:
        f.write(f"TF\tNumber of Target Genes\tTarget Genes\n\n")
        for key, inner_dict in regulon_dict.items():
            f.write(f"{key}\t{len(inner_dict)}\t{list(inner_dict.items())}\n\n")

for i in range(0,20):
    file = f'{scenic_results_dir}/results_step2_5ht_basal/regulons_5ht_basal_run{i}.p'
    output = f'{scenic_results_dir}/results_step2_5ht_basal/Formatted/regulons_5ht_basal_run{i}_formatted.txt'
    save_as_txt(file,output)

# Analyze the files

In [ ]:
def view_reg_dict(regulon_path):
    regulon_dict_without_imp = {}
    regulon_dict_with_imp = {}
    print(regulon_path)
    with open(regulon_path, "rb") as f:
        regulons =pickle.load(f)
    for reg in regulons:
        regulon_dict_without_imp[reg.name] = set([key for key in reg.gene2weight])
        regulon_dict_with_imp[reg.name] = [(key,np.round(reg.gene2weight[key],decimals=3)) for key in reg.gene2weight]
    return regulon_dict_without_imp,regulon_dict_with_imp
    
# for i in range(0,10):
#     f = f'../../results/results_step_2_strat_1_5ht_basal/5ht_basal_regulons_run{i}.p'
    

In [ ]:
dicts = []
dicts_with_imp = []
for i in range(0,20):
    curr_dict,curr_dict_with_imp = view_reg_dict(f'{scenic_results_dir}/results_step2_5ht_basal/regulons_5ht_basal_run{i}.p')
    dicts.append(curr_dict)
    dicts_with_imp.append(curr_dict_with_imp)

# Get Data

In [ ]:
import scanpy as sc
#Get cells and genes used in network analysis
data = sc.read_h5ad(f'{data_dir}/NetworkData_HVGs_basal_5ht6ho_without_PTPRC_Adgre1.h5ad')
ht5_data = data[data.obs['orig.ident'] == '5Ht']
ht5_basal = ht5_data[ht5_data.obs['cluster1'] == 'Mammary epithelial cells-Basal']
tfs = load_tf_names(f'{data_dir}/allTFs_mm.txt')
tfs = sorted(list((set(tfs).intersection(ht5_data.var_names))))
tfs = [tf + '(+)' for tf in tfs]
target_genes = ht5_basal.var_names
cells = ht5_basal.obs_names

#Get normalized but non-scaled data for stats
data = sc.read_h5ad(f'{data_dir}/combined_data_basal_5ht6ho_without_PTPRC_Adgre1_filtered.h5ad')
ht5_basal = data[data.obs_names.isin(cells)]
ht5_basal = ht5_basal[:,list(target_genes)]
counts_df_5ht_basal = pd.DataFrame(ht5_basal.X, index=ht5_basal.obs_names,columns=ht5_basal.var_names)
counts_df_5ht_basal

# Get statistics for each TF and its target genes

In [ ]:
'''
Calculate the following stats for each TF:
1. Frequency of occurence (Number of occurences/total runs)
2. Median of importance scores +/- Standard Deviation of importance scores - from positive runs (calculate this based on 
scores in runs where it actually appears. For example, if gene X appears in 2 out of 20 runs, 
calculate this statistic using scores in those 2 runs.)
3. Median of importance scores +/- Standard Deviation of importance scores - from all runs
4. Counts - mean ;  median +/- standard deviation (Log transform the raw data (filtered after removing bad genes and cells)
and find the counts from there) - these must be calculated for the specific mouse type and cell type (eg 5ht basal).
5. 5. median of its ranks + standard deviation of its ranks (measured by the importance scores in the descending order) 
across all runs in which the target gene is present.
'''
#Loop over all the TFs
for tf in tfs:
    #Make necessary lists to store the results
    all_genes = [] #Will contain all target genes for the current TF across all runs
    ranks = [] # Will contain all target genes,importance and ranks across all runs
    genes_with_ranks = [] #This is a list of lists. Each list within corresponds to a run. Contains target genes,imp and ranks for the current TF
    meds = [] #Will contain stat #2 given above
    sds = []
    meds_all_runs = [] #Will contain stat #3 given above
    sds_all_runs = []
    med_ranks = [] #Will contain stat #5 given above
    sd_ranks = []
    exp_mean = [] #Will contain stat #4 given above
    exp_median = [] #Will contain stat #4 given above
    sd_exp = []
    
    #Loop over regulons in different runs
    #Get a union of all target genes of the current TF across all runs
    for i,d in enumerate(dicts):
        if tf in d:
            all_genes += list(d[tf])
    #Only if this union in non zero, proceed; If zero, the TF is not present in any regulon
    if len(all_genes)!= 0:
        #Calculate the frequency of the target genes.
        # Count the occurrences of each gene
        gene_counts = Counter(all_genes)
        # Calculate the frequency of each gene based on the formula
        n = len(dicts)
        gene_frequencies = {gene: count / n for gene, count in gene_counts.items()}

        # Create a DataFrame from the gene frequencies dictionary
        df = pd.DataFrame(list(gene_frequencies.items()), columns=['Gene', 'Frequency'])
        
        #Loop over regulons containing importance scores.
        for d in dicts_with_imp:
            if tf in d:
                genes_with_ranks.append(list(d[tf]))
        #Get ranks for all target genes, respective to their runs.
        for l in genes_with_ranks:
            sorted_data = sorted(l, key=lambda x: -x[1])
            sorted_data_with_rank = [(item[0], item[1], idx + 1) for idx, item in enumerate(sorted_data)]
            ranks += sorted_data_with_rank
        #Calculate the statistics for each target gene of the current TF
        for gene in df['Gene']:
#             if gene == 'P4ha1':
#                 print(tf,gene)
            curr_imps = []
            curr_imps_succ = []
            curr_ranks = []
            for t in ranks:
                #For each target gene, if it is present in the dataframe (containing frequencies), calculate the stats
                if t[0] == gene:
                    curr_imps.append(t[1])#Save importance
                    curr_ranks.append(t[2])#Save ranks

            curr_imps_succ += curr_imps
            #Add 0 for runs where the current target does not appear
            while len(curr_imps_succ)<len(dicts):
                curr_imps_succ.append(0)
            #Calculate stat #2
            median = statistics.median(curr_imps)
            std_dev = np.std(curr_imps)
            meds.append(np.round(median,3))
            sds.append(np.round(std_dev,3))
            
            #Calculate stat #3
            median_all_run = statistics.median(curr_imps_succ)
            std_dev_all_run = np.std(curr_imps_succ)
            meds_all_runs.append(np.round(median_all_run,3))
            sds_all_runs.append(np.round(std_dev_all_run,3))
            
            #Calculate stat #4
            median_exp = statistics.median(counts_df_5ht_basal[gene])
            std_dev_exp = np.std(counts_df_5ht_basal[gene])
            exp_median.append(np.round(median_exp,3))
            sd_exp.append(np.round(std_dev_exp,3))
            
            #Calculate stat #5
            median_ranks = statistics.median(curr_ranks)
            std_dev_ranks = np.std(curr_ranks)
            med_ranks.append(np.round(median_ranks,3))
            sd_ranks.append(np.round(std_dev_ranks,3))
            exp_mean.append(np.round(np.mean(counts_df_5ht_basal[gene]),3))
        
        #Add these to the dataframe
        df['Median_succ_runs'] = meds  
        df['SD_succ_runs'] = sds
        df['Median_all_runs'] = meds_all_runs
        df['SD_all_runs'] = sds_all_runs
        df['Median_ranks'] = med_ranks
        df['SD_ranks'] = sd_ranks
        df['Mean_expression'] = exp_mean
        df['Median_expression'] = exp_median
        df['SD_expression'] = sd_exp 
        
        df.sort_values(by='Frequency',ascending=False,inplace=True)
        tf_name = tf.split('(')[0]
        temp = [tf,0,0,0,0,0,0,0,np.round(np.mean(counts_df_5ht_basal[tf_name]),3),
                np.round(np.median(counts_df_5ht_basal[tf_name]),3),np.round(np.std(counts_df_5ht_basal[tf_name]),3)]
        temp_df = pd.DataFrame([temp], columns=list(df.columns))
        df = pd.concat([temp_df,df])
        df.reset_index(drop=True, inplace=True)
        
        #Save the dataframe
        df.to_csv(f'{scenic_results_dir}/results_step2_5ht_basal/Stats/{tf}.csv')
    else:#If the TF is not in any regulon in any run, save an empty file.
        df = pd.DataFrame()
        df.to_csv(f'{scenic_results_dir}/results_step2_5ht_basal/Stats/{tf}_empty.csv')